# Title

## Introduction

**Replace the contents of this cell with your own title and introduction.**

Remember that the submitted notebook should form a self-contained document. You do not need to include a full derivation of the numerical model for Newton's law of gravitation, as given in the instructions for the task, but you do need to provide an overview of the relevant physics (including equations) and a discussion of the aims of the investigation and the approach taken.

Your text cells should be structured in complete, grammatically correct, coherent sentences and paragraphs. Do not paste text verbatim from the script.

In [ ]:
from vpython import vector, mag, sphere, rate, color, canvas
G = 1.      # gravitational constant [using units where gravitational constant is 1.0]

## Part A

### 1. Calculating the gravitational force between two objects

You should complete the following function, without changing its name, arguments or docstring, to calculate the force on one object due to the gravitational field of another.

In [ ]:
def force(pos1, pos2, m1, m2):
    """
    Returns the gravitational force exerted by object 2 on object 1.
    Input:
      - pos1 = position vector of first object
      - pos2 = position vector of second object
      - m1   = mass of first object
      - m2   = mass of second object
    Depends on:
      - G    = gravitational constant (global variable)
    """
    ##################
    # YOUR CODE HERE #
    ##################

#### Testing your function

The following cell applies some tests to help you make sure your `force` function works correctly before you use it in the rest of the task. You do not need to understand the details of how it works, but it may help you narrow down any bugs in your code. If each line of output starts with `OK` then it is likely (but not guaranteed) that you have implemented the function correctly.

In [ ]:
################################################
#                                              #
# Test force is correct in a few simple cases. #
#                                              #
#   DO NOT CHANGE THE CODE IN THIS CELL.       #
#                                              #
################################################

def test_force(pos1, pos2, m1, m2, expected_force):
    epsilon = 1e-10
    f = force(pos1, pos2, m1, m2)
    if not isinstance(f,vector):
        print(f"ERROR: function should return a vector but returns {f}.")
        return
    args_as_string = f"({pos1}, {pos2}, {m1}, {m2})"
    error = mag(f-expected_force)
    if error<epsilon:
        print(f"OK: correct results for input {args_as_string}")
    else:
        print(f"ERROR: wrong results for input {args_as_string}")
        print(f"  expected: {expected_force}")
        print(f"  got:      {f}")

test_force(vector(0,0,0),vector(1,0,0),1,1,vector(1,0,0))    # distance = 1 in x direction
test_force(vector(1,0,0),vector(0,0,0),1,1,vector(-1,0,0))   # swap objects
test_force(vector(0,0,0),vector(2,0,0),1,1,vector(0.25,0,0)) # distance = 2
test_force(vector(0,0,0),vector(0,1,0),1,1,vector(0,1,0))    # distance = 1 in y direction
test_force(vector(10,0,0),vector(10,1,0),1,1,vector(0,1,0))  # displaced from origin

### 2. Calculating the motion of a planet in the gravitational field of a star

You should complete the following function, without changing its name, arguments or docstring, to calculate the new position and velocity of a planet after a time step `dt`, in the gravitational field of a star of a given mass situated at the origin. This function will need to call `force` to calculate the acceleration vector of the planet.

In [ ]:
def move_planet(position, velocity, m_star, dt):
    """
    Calculate motion of planet in the gravitational field of a star with given mass
    at the origin, using Euler's method.
    
    Input:
      - position: position vector of planet at start of time step
      - velocity: velocity vector of planet at start of time step
      - m_star:   mass of star
      - dt:       time step
      
    Output: (position_new, velocity_new)
      - position_new: position vector of planet at end of time step
      - velocity_new: velocity vector of planet at end of time step
      
    Depends on:
      - force = function to calculate the gravitational force between two objects
    """
    ##################
    # YOUR CODE HERE #
    ##################

#### Testing your function

The following cell applies some tests to help you make sure your `move_planet` function works correctly before you use it in the rest of the task. You do not need to understand the details of how it works, but it may help you narrow down any bugs in your code. If each line of output starts with `OK` then it is likely (but not guaranteed) that you have implemented the function correctly.

In [ ]:
######################################################
#                                                    #
# Test move_planet is correct in a few simple cases. #
#                                                    #
#   DO NOT CHANGE THE CODE IN THIS CELL.             #
#                                                    #
######################################################

def test_move_planet(position, velocity, m_star, dt, expected_pos, expected_vel):
    epsilon = 1e-10
    results = move_planet(position, velocity, m_star, dt)
    if not isinstance(results, tuple):
        print(f"ERROR: function should return two vectors but returns {results}.")
        return
    if not len(results)==2:
        print(f"ERROR: function should return two vectors but returns {results}.")
        return
    pos, vel = results
    if not (isinstance(pos,vector) and isinstance(vel,vector)):
        print(f"ERROR: function should return two vectors but returns {results}.")
        return
    args_as_string = f"{position}, {velocity}, {m_star}, {dt}"
    err_pos, err_vel = mag(pos - expected_pos), mag(vel - expected_vel)
    if err_pos < epsilon and err_vel < epsilon:
        print(f"OK: correct results for input {args_as_string}")
    else:
        print(f"ERROR: wrong results for input {args_as_string}")
        print(f"  expected: {expected_pos, expected_vel}")
        print(f"  got:      {results}")


test_move_planet(vector(1,0,0), vector(1,0,0), 1, 0, vector(1,0,0), vector(1,0,0))   # dt = 0: output = input
test_move_planet(vector(1,0,0), vector(1,0,0), 1, 1, vector(2,0,0), vector(0,0,0))   # moving away from star
test_move_planet(vector(0,1,0), vector(0,-1,0), 1, 1, vector(0,0,0), vector(0,-2,0)) # moving towards star
test_move_planet(vector(1,0,0), vector(0,1,0), 1, 1, vector(1,1,0), vector(-1,1,0))  # moving past star
test_move_planet(vector(1,0,0), vector(0,1,0), 1, 0.1, vector(1,0.1,0), vector(-0.1,1,0))  # smaller dt
test_move_planet(vector(1,0,0), vector(0,1,0), 2, 0.1, vector(1,0.1,0), vector(-0.2,1,0))  # larger star mass

### 3. Animating the orbit of a planet

You should complete the following function, without changing its name, arguments or docstring, to display an animation of the orbit of the planet with the given intial position and velocity, and the given time step. Your planet should leave a visible trail so that we can see the shape of path.

In [ ]:
def animate_planet(position, velocity, m_star, dt):
    """
    Animate planetary orbit from given starting position, with given time step.
    """
    ##################
    # YOUR CODE HERE #
    ##################

#### Testing your function

If you have implemented all three functions correctly, the following cell should show an almost elliptical orbit.

In [ ]:
# Initialize canvas, and set parameters of star and planet.
canvas()
pos_planet = vector(0,2,0)   # initial position of planet
v_planet   = vector(-22,0,0) # initial velocity of planet
m_star     = 1000.           # mass of star (units where G=1)
sphere(pos=vector(0,0,0), color=color.yellow, radius=0.1) # draw star

# Animate orbit of planet
animate_planet(pos_planet, v_planet, m_star, 1e-4)

### 4. Investigation

You should insert code and text cells below as required to investigate and discuss the effect of changing the parameters of the animation: time step, mass of star, initial position and velocity of planet. 

## Part B

It is up to you to structure the rest of the notebook as you see fit, as you complete the tasks set in part B of the assignment.

You can call the functions you have defined in part A, or copy and adapt them in part B, but **DO NOT CHANGE THE CODE IN PART A SUCH THAT THE CELLS IN PART A NO LONGER WORK CORRECTLY.** You do not want to lose marks in section A in completing section B.